In [1]:
import sys
import time
import bibtexparser
import itertools
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
import bibtexparser
bibtex_file = open('gap-publishednicer.bib.txt', encoding='utf-8')
bib_data = bibtexparser.load(bibtex_file)

In [5]:
bib = bib_data.entries

In [10]:
final_df = pd.read_csv('local_test_output.csv', dtype='str')

In [15]:
final_df

,MR,Citation
0,MR4044696,"The GAP Group, GAP – groups, algorithms and pr..."
1,MR2900886,"The GAP Group, $GAP$ groups, algorithms, and p..."
2,MR3169623,"Distler, A., Mitchell, J. D. (2011). Smallsemi..."
3,MR3169623,"The GAP Group, (2008). (http://www.gap-system...."
4,MR4180136,"The GAP Group, 2019. GAP – Groups, Algorithms,..."


In [11]:
mrs = final_df['MR']

In [12]:
print(type(bib))
print(type(bib[5]))
print(bib[5])

<class 'list'>
<class 'dict'>
{'printedkey': 'A01', 'mrreviewer': 'H. Heineken', 'mrnumber': '1864795', 'mrclass': '20F45 (20D60 20F19)', 'issn': '0362-1588', 'fjournal': 'Houston Journal of Mathematics', 'pages': '511–522', 'year': '2001', 'number': '3', 'volume': '27', 'journal': 'Houston J. Math.', 'title': 'Some {E}ngel conditions on finite subsets of certain groups', 'author': 'Abdollahi, A.', 'ENTRYTYPE': 'article', 'ID': 'MR1864795'}


In [13]:
bib[5]

{'printedkey': 'A01',
 'mrreviewer': 'H. Heineken',
 'mrnumber': '1864795',
 'mrclass': '20F45 (20D60 20F19)',
 'issn': '0362-1588',
 'fjournal': 'Houston Journal of Mathematics',
 'pages': '511–522',
 'year': '2001',
 'number': '3',
 'volume': '27',
 'journal': 'Houston J. Math.',
 'title': 'Some {E}ngel conditions on finite subsets of certain groups',
 'author': 'Abdollahi, A.',
 'ENTRYTYPE': 'article',
 'ID': 'MR1864795'}

In [14]:
bib[5].keys()
bib[5]['year']
bib[5]['fjournal']

'Houston Journal of Mathematics'

In [42]:
years = []
journals = []

for mrs in bib:
    if mrs == bib[i]['ID']:
        year = bib[i]['fjournal']
        
    journals.append(year)
   

NameError: name 'i' is not defined

In [56]:
bib_mr = ['MR' + str(d['mrnumber'] for d in bib if 'mrnumber' in d)]
journal = [d['fjournal'] for d in bib if 'fjournal' in d]
year =  [d['year'] for d in bib if 'year' in d]
pub_type =  [d['ENTRYTYPE'] for d in bib if 'ENTRYTYPE' in d]
author = [d['author'] for d in bib if 'author' in d]


In [58]:
print(journal)

['Theoretical Computer Science', 'Revista de la Real Academia de Ciencias Exactas, Físicas y Naturales. Serie A. Matematicas. RACSAM', 'Communications in Algebra', 'Journal of Pure and Applied Algebra', 'Houston Journal of Mathematics', 'Journal of Algebra', 'Linear Algebra and its Applications', 'Journal of Algebra', 'Journal of Group Theory', 'Journal of Algebra and its Applications', 'Bulletin of the Malaysian Mathematical Sciences Society. Second Series', 'Communications in Algebra', 'Communications in Algebra', 'Algebra Colloquium', 'Journal of Algebra', 'International Journal of Algebra and Computation', 'Communications in Algebra', 'Archiv der Mathematik', 'Communications in Algebra', 'Communications in Algebra', 'Communications in Algebra', 'Journal of Algebra and its Applications', 'Communications in Algebra', 'Journal of Algebra and its Applications', 'Journal of Pure and Applied Algebra', 'Houston Journal of Mathematics', 'Journal of Algebra and its Applications', 'European 

In [47]:
values_of_key = [a_dict["journal"] for a_dict in bib]

KeyError: 'journal'

In [48]:
values_of_key

['Abas, M. and Vetrík, T.',
 'Abbas, A. and Assi, A. and García-Sánchez, P. A.',
 'Abdeljaouad, I.',
 'Abdolghafourian, A. and Iranmanesh, M. A.',
 'Abdolghafourian, A. and Iranmanesh, M. A. and Niemeyer, A. C.',
 'Abdollahi, A.',
 'Abdollahi, A.',
 'Abdollahi, A.',
 'Abdollahi, A.',
 'Abdollahi, A.',
 'Abdollahi, A. and Ahmadi, M. and Ghoraishi, S. M.',
 'Abdollahi, A. and Ashraf, F. and Shaker, S. M.',
 'Abdollahi, A. and Ataei, M. J. and Hassanabadi, A. M.',
 'Abdollahi, A. and Ataei, M. J. and Jafarian Amiri, S. M. and Hassanabadi, A. M.',
 'Abdollahi, A. and Azad, A. and Mohammadi Hassanabadi, A. and Zarrin, M.',
 'Abdollahi, A. and Brandl, R. and Tortora, A.',
 'Abdollahi, A. and Daoud, B. and Farrokhi D. G. , M. and Guerboussa, Y.',
 'Abdollahi, A. and Faghihi, A. and Hassanabadi, A. M.',
 "Abdollahi, A. and Faghihi, A. and Linton, S. A. and O'Brien, E. A.",
 'Abdollahi, A. and Faghihi, A. and Mohammadi Hassanabadi, A.',
 'Abdollahi, A. and Guedri, M. and Guerboussa, Y.',
 'Abdo